# Loading a PDB File and Visualizing in a Notebook
Jupyter Notebook is a versatile open-source web application that allows for the creation and sharing of documents containing live code, equations, visualizations, and narrative text. It's widely used in data science and scientific computing but is equally suited to general programming tasks. Notebooks are composed of individual cells that can hold code, text, or markdown. These cells can be run independently, which means you can execute specific sections of code and observe the results in real-time. This encourages hands-on exploration and data-driven experimentation.

In this Jupyter Notebook, we'll use OpenMM and NGLview to load and visualize protein structures. Notebooks provide an interactive environment that is flexible and conducive to experimentation.

## NGLview
NGLview is a browser-based molecular viewer that offers a convenient way to inspect changes we make to PDB files or molecular structures. Although its functionality is somewhat limited, NGLview is immensely helpful in understanding structures before devoting significant resources to running simulations.

The first cell below loads the protein kinase-inhibitor complex 5CEO. In this instance, we're retrieving the protein structure from a local file bundled with this notebook. However, it's important to note that NGLview can directly fetch structures from the RCSB Protein Data Bank as well.

In [1]:
import nglview as nv
view = nv.show_structure_file("assets/cookbook/5ceo.pdb")
# Which for PDBs already on the RSDB database is the same as
view = nv.show_pdbid("5ceo")
view

NGLWidget()

### Changing how the ligand looks
The bellow command makes the ligand spacefill VDW, in the above cell

In [ ]:
view.add_representation("spacefill","ligand")

# Changing how the protein looks

Similar to how Nanome can change the rendering of a protine or ligand we can do the same with NGLview. The below code adds a surface to the view and makes the rendering semitransparent.

In [2]:
view.clear_representations()
view.add_cartoon()
view.add_surface('.CA', opacity=0.3)
#view.add_licorice('not hydrogen')

### Going back to default view

In [ ]:
view.clear_representations()
view.add_representation("ribbon",color_scheme="residueindex")
view.add_ball_and_stick("ligand")

### What colour schemes can we pick from?

In [ ]:
print(nv.color.COLOR_SCHEMES[1:])
# We can pick the name of a sting from any of the blow generated list

### Playing back a molecular dynamics trajectory from a file

There are a number of ways of viewing trajectories in NGLview, but bellow is one of the simplest ways by using simpletraj. In this case the topology or "connectivity" are held in the gro file and the  coordinates are held in the xtc file, which is structured similar to a movie file, with each frame being a step in the MD simulation.

In [2]:
traj = nv.SimpletrajTrajectory("assets/cookbook/6OIM_traj.xtc", "assets/cookbook/6OIM_coord.gro")
print(f"Trajectory has {traj.n_frames} frames")
viewtraj = nv.show_simpletraj(traj)
viewtraj 

Trajectory has 201 frames


NGLWidget(max_frame=200)

In [1]:
from nanome.beta.redis_interface import PluginInstanceRedisInterface
import os

# Set up redis credentials
redis_host = os.environ.get("REDIS_HOST")
redis_port = os.environ.get("REDIS_PORT")
redis_password = os.environ.get("REDIS_PASSWORD")
redis_channel = os.environ.get("REDIS_CHANNEL")
print(redis_host, redis_port, redis_password,redis_channel)

plugin = PluginInstanceRedisInterface(redis_host, redis_port, redis_password, redis_channel)
plugin.connect()
print('Plugin Connected')

redis 6379 None YLAC8StAPiBtsJf6


TimeoutError: Timeout waiting for response from RPC get_plugin_data

In [ ]:
# Add original pdb to workspace
pdb_file = "assets/cookbook/6OIM.pdb"
plugin.send_files_to_load([pdb_file])

In [ ]:
# Get complex
comp = plugin.request_workspace().complexes[-1]

In [ ]:
# Setup stream to update atom positions on the fly.
from nanome.util import enums

stream_type = enums.StreamType.position
atom_indices = [atm.index for atm in comp.atoms]
strm = plugin.create_writing_stream(atom_indices, stream_type)

In [ ]:
# Loop through all the frames and render positions in Nanome.
frame_count = traj.n_frames
for i in range(0, frame_count):
    # print(f"Frame {i+1} of {frame_count}")
    coords = traj.get_coordinates(i)
    stream_data = []
    for coord in coords:
        stream_data.extend(coord)
    plugin.update_stream(strm, stream_data)